In [346]:
import wandb
import pandas as pd
import numpy as np
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Whisker
from bokeh.transform import factor_cmap, jitter
from bokeh.layouts import gridplot

from fundusClassif.data.data_factory import get_datamodule_from_config
from nntools.utils import Config


In [347]:
api = wandb.Api()
runs = api.runs("hmr24/FundusDRGrading-src_fundusClassif_scripts")

important_keys = ['lr','preprocessing', 'data_augmentation_type']
list_configs = []
for i, r in enumerate(runs):
    if i < 74:
        pass
    else :
        kappa = r.summary_metrics.get("Validation Quadratic Kappa", None)
        kappa_ddr = r.summary_metrics.get("Quadratic Kappa_DDR_test", None)
        kappa_eyepacs = r.summary_metrics.get("Quadratic Kappa_EYEPACS_test", None)
        kappa_idrid = r.summary_metrics.get("Quadratic Kappa_IDRID_test", None)
        if kappa is None:
            continue
        config = {k: v for k, v in r.config.items() if k in important_keys}
        config['name'] = r.name
        config['kappa'] = kappa
        config['kappa_ddr'] = kappa_ddr
        config['kappa_eyepacs'] = kappa_eyepacs
        config['kappa_idrid'] = kappa_idrid
    
        list_configs.append(config)

df = pd.DataFrame(list_configs)
df

,lr,preprocessing,data_augmentation_type,name,kappa,kappa_ddr,kappa_eyepacs,kappa_idrid
0,0.00100,sarki,None,upbeat-sweep-1,0.932644,0.658787,0.616012,0.688647
1,0.00100,absent,None,sweepy-sweep-2,0.879902,0.573273,0.478262,0.637572
2,0.00100,autobalance,None,royal-sweep-3,0.897062,0.623832,0.531265,0.662328
3,0.00100,clahe_lab,None,dulcet-sweep-4,0.922191,0.648264,0.567507,0.586553
4,0.00100,clahe_rgb,None,amber-sweep-5,0.923834,0.654336,0.614074,0.717875
...,...,...,...,...,...,...,...,...
65,0.00001,autobalance,light,proud-sweep-66,0.940386,0.699467,0.673494,0.730624
66,0.00001,clahe_lab,light,graceful-sweep-67,0.945784,0.729886,0.676506,0.669743
67,0.00001,clahe_rgb,light,lucky-sweep-68,0.945258,0.740294,0.697102,0.766788
68,0.00001,clahe_max_green_gsc,light,light-sweep-69,0.944662,0.752400,0.691975,0.726111


In [348]:
df_preprocessing_type = df[df['preprocessing'] == 'absent']

config = Config('../configs/config.yaml')
datamodule = get_datamodule_from_config(config['datasets'], config['data'])
test_dataloader = datamodule.test_dataloader()
test_datasets_ids = [d.dataset.id for i, d in enumerate(test_dataloader)]
dataloader0_weight = 0
dataloader1_weight = 0
dataloader2_weight = 0
lenght_total_test = 0
for i, d in enumerate(test_dataloader):
    lenght_total_test += len(d.dataset)
#print(lenght_total_test)

for i, d in enumerate(test_dataloader):
    if d.dataset.id == test_datasets_ids[0]:
        dataloader0_weight = len(d.dataset)/lenght_total_test
        #print(dataloader0_weight)
        #print(d.dataset.id)
    elif d.dataset.id == test_datasets_ids[1]:
        dataloader1_weight += len(d.dataset)/lenght_total_test
        #print(dataloader1_weight)
        #print(d.dataset.id)
    elif d.dataset.id == test_datasets_ids[2]:
        dataloader2_weight += len(d.dataset)/lenght_total_test
        #print(dataloader2_weight)
        #print(d.dataset.id)

df_preprocessing_type['kappa_weighted'] = 0 
for i in range(len(df_preprocessing_type)):
    eyepacs = df_preprocessing_type['kappa_eyepacs'].iloc[i]*dataloader0_weight
    idrid = df_preprocessing_type['kappa_idrid'].iloc[i]*dataloader1_weight
    ddr = df_preprocessing_type['kappa_ddr'].iloc[i]*dataloader2_weight
    df_preprocessing_type['kappa_weighted'].iloc[i] =  eyepacs + idrid + ddr
df_preprocessing_type



Train 28101 Val 7025 Test 53575
Train 2930 Val 732 <fundus_data_toolkit.datamodules.classification.IDRiDDataModule object at 0x7c0fde76f8c0>
Train 331 Val 82 Test 103
Train 6260 Val 2503 Test 3759


/home/florian/miniconda3/envs/DL/lib/python3.12/site-packages/nntools/dataset/classif_dataset.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(self.file_column, inplace=True)
/tmp/ipykernel_326460/2424636022.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_preprocessing_type['kappa_weighted'] = 0
/tmp/ipykernel_326460/2424636022.py:34: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which wil

,lr,preprocessing,data_augmentation_type,name,kappa,kappa_ddr,kappa_eyepacs,kappa_idrid,kappa_weighted
1,0.00100,absent,None,sweepy-sweep-2,0.879902,0.573273,0.478262,0.637572,0.484766
8,0.00050,absent,None,woven-sweep-9,0.938206,0.702810,0.638068,0.680533,0.642381
15,0.00010,absent,None,blooming-sweep-16,0.937450,0.710509,0.685425,0.690614,0.687076
22,0.00005,absent,None,floral-sweep-23,0.936039,0.701023,0.650783,0.622903,0.654021
29,0.00001,absent,None,legendary-sweep-30,0.935222,0.662668,0.580488,0.632846,0.585960
36,0.00100,absent,light,fragrant-sweep-37,0.741907,0.737429,0.594548,0.603253,0.603914
43,0.00050,absent,light,treasured-sweep-44,0.832430,0.726600,0.691384,0.770060,0.693830
50,0.00010,absent,light,comfy-sweep-51,0.912008,0.775938,0.728072,0.739399,0.731225
57,0.00005,absent,light,leafy-sweep-58,0.931236,0.774046,0.724562,0.752465,0.727850
64,0.00001,absent,light,absurd-sweep-65,0.952053,0.752305,0.691079,0.715029,0.695129
